In [1]:
!pip install pymysql
!pip install pymssql

import LibData
import LibUtils
import LibML
import LibFile
import LibEE

import pandas as pd
import numpy as np


     |████████████████████████████████| 51kB 3.2MB/s 
     |████████████████████████████████| 1.3MB 7.7MB/s 


In [2]:

jobName = '201216_Train_split1'

# 단계1. 베이스 : 원핫 -> 스케일러
x1, y1 = LibData.GetXY_s_o(jobName)

# end
# 단계6. 임베딩2 :  원핫 -> 머니니스 -> 가입나이 -> 금리축소 -> 임베딩_상품코드 -> 임베딩_경과기간x현재나이 -> 스케일러
x2, y2 = LibData.GetXYBase(jobName)
x2 = LibData.GetMoneyness(x2)
x2 = LibData.GetAttainedAge(x2)
x2 = LibData.GetIrReduction(x2)
x2 = LibEE.GetXIntersected(x2, ['ElapsedMth','AttainedAge'], 'Inter_EM_AA')
x2, emb = LibEE.GetXEmbApplied(3, x2, y2, ['ProdCode', 'Inter_EM_AA'])
scaler, x2 = LibUtils.ScaleStandard(x2)

colsSelected = ['ElapsedMth', 'Sex', 'Prem', 'PremYr', 'AccumPrem', 'ContAmt', 'StartAge', 'FundAllo03', 'IR06', 'IR09', 'IR12', 'IR14', 'Moneyness', 'AttainedAge', 'ee_ProdCode_00', 'ee_ProdCode_01', 'ee_ProdCode_02']
x2 = x2[colsSelected]



In [ ]:
#SVM
from sklearn.svm import SVR

# 시작점
# model = SVR(kernel = 'rbf', C=50, gamma=0.1)
# print(LibML.GetScoresCV(x1,y1,model,5))
# [331.2175, 0.3963, 0.5907]

# 튜닝전
model = SVR(kernel = 'rbf', C=50, gamma=0.1)
print(LibML.GetScoresCV(x2,y2, model,5))
# ???

# 튜닝후
# model = SVR(kernel = 'rbf', C = 184, gamma = 0.162)
# print(LibML.GetScoresCV(x2,y2,model,5))
# [162.33, 0.1944, 0.8468]

start learning 1 2021-01-17 14:16... [218.03, 0.2607, 0.7527]... end learning 2021-01-17 15:20
start learning 2 2021-01-17 15:20... [219.0, 0.26, 0.7636]... end learning 2021-01-17 16:24
start learning 3 2021-01-17 16:24... [211.24, 0.2527, 0.7914]... end learning 2021-01-17 17:28
start learning 4 2021-01-17 17:28... [219.49, 0.261, 0.7709]... end learning 2021-01-17 18:31
start learning 5 2021-01-17 18:31... 

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge, Lasso

x1_poly = pd.DataFrame(PolynomialFeatures(2).fit_transform(x1))
x2_poly = pd.DataFrame(PolynomialFeatures(2).fit_transform(x2))

In [ ]:
# Ridge

model1 = Ridge()
print(LibML.GetScoresCV(x1_poly, y1, model1, 5))
# [326.95, 0.3909, 0.8148].... 이상

model1 = Ridge()
print(LibML.GetScoresCV(x2_poly, y2, model1, 5))
# [329.5, 0.394, 0.8107]

model2 = Ridge(alpha = 0.147)     #3954
print(LibML.GetScoresCV(x2_poly, y2, model2, 5))
# [329.64, 0.3942, 0.8105]

start learning 1 2021-01-17 14:04... [325.5, 0.3889, 0.8184]... end learning 2021-01-17 14:04
start learning 2 2021-01-17 14:04... [329.57, 0.393, 0.8085]... end learning 2021-01-17 14:05
start learning 3 2021-01-17 14:05... [326.4, 0.3981, 0.8114]... end learning 2021-01-17 14:05
start learning 4 2021-01-17 14:05... [326.09, 0.3838, 0.8203]... end learning 2021-01-17 14:05
start learning 5 2021-01-17 14:05... [327.2, 0.391, 0.8154]... end learning 2021-01-17 14:05
[326.95, 0.391, 0.8148]
start learning 1 2021-01-17 14:05... [328.25, 0.3934, 0.8144]... end learning 2021-01-17 14:05
start learning 2 2021-01-17 14:05... [330.45, 0.3975, 0.808]... end learning 2021-01-17 14:05
start learning 3 2021-01-17 14:05... [327.98, 0.3937, 0.8103]... end learning 2021-01-17 14:05
start learning 4 2021-01-17 14:05... [328.56, 0.3917, 0.8138]... end learning 2021-01-17 14:05
start learning 5 2021-01-17 14:05... [332.53, 0.3938, 0.8062]... end learning 2021-01-17 14:05
[329.55, 0.394, 0.8105]
start le

In [ ]:
# Lasso

model = Lasso()    
print(LibML.GetScoresCV(x1_poly, y1, model, 5))
# [340.68, 0.4073, 0.7958]

model = Lasso()
print(LibML.GetScoresCV(x2_poly, y2, model, 5))
# [331.04, 0.3958, 0.8068]

model = Lasso(alpha = 0.185)    
print(LibML.GetScoresCV(x2_poly, y2, model, 5))
# [329.64, 0.3941, 0.8098]

start learning 1 2021-01-17 13:37... 

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 8884387.549205303, tolerance: 2168365.162065795
  positive)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 837415197.8598938, tolerance: 11559130.454511665
  positive)


[342.48, 0.4104, 0.789]... end learning 2021-01-17 13:40
start learning 2 2021-01-17 13:40... 

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 9063284.785814524, tolerance: 2141982.792685969
  positive)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 824384216.4939613, tolerance: 11421304.437801419
  positive)


[341.45, 0.4039, 0.7972]... end learning 2021-01-17 13:43
start learning 3 2021-01-17 13:43... 

...

[331.84, 0.394, 0.8087]... end learning 2021-01-17 13:59
start learning 4 2021-01-17 13:59... 

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 27956514.21166289, tolerance: 2142144.8695909153
  positive)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 294778542.7296295, tolerance: 11554832.334242009
  positive)


[333.43, 0.4005, 0.8042]... end learning 2021-01-17 14:00
start learning 5 2021-01-17 14:00... 

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 29389300.35479617, tolerance: 2156191.184965229
  positive)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 263058121.46548843, tolerance: 11503087.92794325
  positive)


[326.9, 0.3931, 0.8154]... end learning 2021-01-17 14:01
[329.64, 0.3941, 0.8098]
